This is about  Titanic ship disaster ,who were survived 


In [0]:
#Importing all the needed libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

In [411]:
#reading the training and testing files
train = pd.read_csv("/train.csv")
test = pd.read_csv("/test.csv")
#print the info of training dataset
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [412]:
train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [0]:
def get_max_hist_val(pc,ag):
  
  #find the histogram for age with respect to pclass
  x1=[]
  x2=[]
  x3=[]
  Pclass = pc
  Age = ag
  
  for cnt,pcag in enumerate(zip(Pclass,Age,np.isfinite(Age))):
    if(pcag[2]):
      if(pcag[0]==1):
        x1.append(pcag[1])
      elif (pcag[0]==2):
        x2.append(pcag[1])
      else:
        x3.append(pcag[1]) 


  hists=[np.histogram(x1),np.histogram(x2),np.histogram(x3)]

  index_max = [np.argmax(hists[0][0]),np.argmax(hists[1][0]),np.argmax(hists[2][0])]

  x1_max_cnt_age = [hists[0][1][index_max[0]],hists[1][1][index_max[1]],hists[2][1][index_max[2]]]
  return x1_max_cnt_age

ages_hists=get_max_hist_val(train.Pclass,train.Age)


In [0]:
def update_age1(columns):
  age=columns[0]
  pclass=columns[1]
  if(pd.isnull(age)):
    if(pclass==1):
      return ages_hists[0]
    elif(pclass==2):
      return ages_hists[1]
    else:
      return ages_hists[2]
  else:
      return age

In [0]:
train['Age']=train[['Age', 'Pclass']].apply(update_age1, axis=1)

In [0]:
#Dropping the Cabin column
train.drop('Cabin', axis=1, inplace=True)
#Dropping the remaining rows with null values in them
train.dropna(inplace=True)


In [0]:
#Categorize the colums data

pclass = pd.get_dummies(train['Pclass'],drop_first=True)
sex = pd.get_dummies(train['Sex'],drop_first=True)
embarked = pd.get_dummies(train['Embarked'],drop_first=True)

#Combine the categorized colums with actual colums

train=pd.concat([train,sex,embarked,pclass], axis=1)

In [0]:
#Drop the colums which are used for categorization
train.drop(['PassengerId','Pclass','Sex','Embarked','Ticket','Name'],axis=1,inplace=True)
#claening process for the training data is done


In [0]:
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())

In [420]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           418 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [0]:
#update the Age parameter
ages_hists=get_max_hist_val(test.Pclass,test.Age)
test['Age']=test[['Age', 'Pclass']].apply(update_age1, axis=1)



In [0]:
#Dropping the Cabin column
test.drop('Cabin', axis=1, inplace=True)
#Dropping the remaining rows with null values in them
test.dropna(inplace=True)

In [0]:
#Categorize the colums data

pclass = pd.get_dummies(test['Pclass'],drop_first=True)
sex = pd.get_dummies(test['Sex'],drop_first=True)
embarked = pd.get_dummies(test['Embarked'],drop_first=True)

#Combine the categorized colums with actual colums

test=pd.concat([test,sex,embarked,pclass], axis=1)

In [0]:
test_ids=test.iloc[:,0]
#Drop the colums which are used for categorization
test.drop(['PassengerId','Pclass','Sex','Embarked','Ticket','Name'],axis=1,inplace=True)



In [0]:
#Prepare the training and testing sets
#
x_train = train.iloc[:,1:]
y_train = train.iloc[:,0:1]
x_test = test.iloc[:,:]




In [426]:
#Training and Predicting

#logistic Regression
from sklearn.linear_model import LogisticRegression
logisticReg=LogisticRegression()
logisticReg.fit(x_train,y_train)

y_pred= logisticReg.predict(x_test)
#accuracy
accuracy = round(logisticReg.score(x_train, y_train) * 100, 2)

#write them into the .csv
df=pd.DataFrame(dict(PassengerId = test_ids, Survived = y_pred)).reset_index()
df.drop('index', axis=1, inplace=True)
df.to_csv('/logresult.csv', index=False)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [427]:
#Random foreset for training

from sklearn.ensemble import RandomForestClassifier
ranFor = RandomForestClassifier(n_estimators = 70)
ranFor.fit(x_train,y_train)
y_pred2= ranFor.predict(x_test)
accuracy2 =round(ranFor.score(x_train, y_train)*100,2)
df=pd.DataFrame(dict(PassengerId = test_ids, Survived = y_pred2)).reset_index()
df.drop('index', axis=1, inplace=True)

df.to_csv('/rfcresult.csv', index=False)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [428]:
#Support vector machine

from sklearn.svm import SVC
svc=SVC()
svc.fit(x_train, y_train)

y_pred3=svc.predict(x_test)
accuracy3=round(svc.score(x_train, y_train)*100,2)
print(accuracy3)
df=pd.DataFrame(dict(PassengerId = test_ids, Survived = y_pred2)).reset_index()
df.drop('index', axis=1, inplace=True)

df.to_csv('/svcresult.csv', index=False)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


88.08
